In [1]:
from keras.optimizers import *
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils import np_utils

import os,sys
import numpy as np
import pandas as pd
import cv2,time,glob,math
import matplotlib.pyplot as plt # plt 用于显示图片
from tqdm import tqdm #进度条

Using TensorFlow backend.


In [2]:
#设置一些基本参数
train_dir = 'train'
valid_dir = 'validation'
test_dir = 'test2'
nb_train_samples = sum([len(x) for _, _, x in os.walk(train_dir)])
nb_valid_samples = sum([len(x) for _, _, x in os.walk(valid_dir)])
nb_test_samples = sum([len(x) for _, _, x in os.walk(test_dir)])
nb_categories = 10
epochs = 5
labels = [" normal driving","texting - right","talking on the phone - right","texting - left",
          "talking on the phone - left","operating the radio",
          "drinking"," reaching behind","hair and makeup","talking to passenger"]

In [11]:
#set training parameters
model_name = 'ResNet50'
fine_tune_layer = 0
optimizer = 'Adam'
learning_rate = 1e-6
augmentation = 'yes'
valid_set = 'p014_p039'
batch_size = 16
img_width, img_height = 400, 300

In [5]:
# 利用Keras的ImageDataGenerator对图片进行预处理
def preprocess_input(x):
    x = x[..., ::-1]
    x = x - [103.939, 116.779, 123.68]
    return x

train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=15) 
#考虑到体型、姿势略有不同，设置少许的旋转、放大缩小和平移。不做翻转） 

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#验证集和测试集无须进行上述处理，保留原样

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(img_width,img_height),  
        shuffle=True,
        batch_size=batch_size,
        class_mode='categorical')  


validation_generator = datagen.flow_from_directory(
        valid_dir,
        target_size=(img_width,img_height),
        shuffle=True,
        batch_size=batch_size,
        class_mode='categorical')

test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=batch_size,
        class_mode='categorical')

Found 20897 images belonging to 10 classes.
Found 1527 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [6]:
from keras import backend as K
K.clear_session()

inputs = Input(shape=(img_width, img_height, 3))
base_model = ResNet50(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print 'total layer count: ', len(base_model.layers),
print '| fine tune layer: ', fine_tune_layer

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

total layer count:  175 | fine tune layer:  0


In [13]:
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
result = model.fit_generator(train_generator,
                   steps_per_epoch=nb_train_samples//batch_size+1,
                   epochs=1,
                   validation_data=validation_generator,
                   validation_steps=nb_valid_samples//batch_size+1)
model.save_weights('models/ResNet50_400_300_p014_p039_l0.h5')

Epoch 1/1
 211/1307 [===>..........................] - ETA: 1509s - loss: 0.0167 - acc: 0.9947

KeyboardInterrupt: 

In [19]:
#write results, use'a' to countinue writing
import csv
print 'model  | layer|  opt  |  l-rate |  aug  |    driver   |batch | epo |  t_acc  |  t_loss |  v_acc  | v_loss '
print '----------------------------------------------------------------------------------------------------------'
with open("result201102.csv","a") as csvfile: 
    writer = csv.writer(csvfile)
    for i in range(len(result.epoch)):
        writer.writerow([model_name, fine_tune_layer, optimizer, learning_rate, augmentation, 
                         valid_set, batch_size, result.epoch[i], '%.3f' % result.history['acc'][i],  
                         '%.3f' % result.history['loss'][i], '%.3f' % result.history['val_acc'][i],
                         '%.3f' % result.history['val_loss'][i]])
        print model_name,' | ', fine_tune_layer,' | ', optimizer, ' | ',learning_rate,' | ', augmentation,
        print ' | ',valid_set, ' | ', batch_size, ' | ', result.epoch[i], ' | ', 
        print '%.3f' % result.history['acc'][i],' | ','%.3f' % result.history['loss'][i], ' | ',
        print '%.3f' % result.history['val_acc'][i],' | ','%.3f' % result.history['val_loss'][i]

model  | layer|  opt  |  l-rate |  aug  |    driver   |batch | epo |  t_acc  |  t_loss |  v_acc  | v_loss 
----------------------------------------------------------------------------------------------------------
ResNet50  |  0  |  Adam  |  0.0001  |  yes  |  p014_p039  |  16  |  0  |  0.903  |  0.312  |  0.912  |  0.260


In [20]:
#load csv and show results
import csv
with open('result201102.csv',"r") as csvfile:
    reader = csv.reader(csvfile)
    #这里不需要readlines
    for line in reader:
        for i in range(len(line)):
            print line[i]," | ",
        print

Xception  |  0  |  Adam  |  0.0001  |  yes  |  p045_p047  |  16  |  0  |  0.925  |  0.251  |  0.887  |  0.428  | 
Xception  |  0  |  Adam  |  1e-05  |  yes  |  p045_p047  |  16  |  0  |  0.993  |  0.028  |  0.904  |  0.453  | 
Xception  |  126  |  Adam  |  1e-05  |  yes  |  p045_p047  |  64  |  0  |  0.623  |  1.294  |  0.743  |  0.751  | 
Xception  |  126  |  Adam  |  1e-05  |  yes  |  p045_p047  |  64  |  0  |  0.874  |  0.544  |  0.748  |  0.727  | 
Xception  |  66  |  Adam  |  0.0001  |  yes  |  p045_p047  |  64  |  0  |  0.940  |  0.241  |  0.838  |  0.445  | 
Xception  |  66  |  Adam  |  1e-06  |  yes  |  p045_p047  |  64  |  0  |  0.991  |  0.037  |  0.883  |  0.365  | 
Xception  |  66  |  Adam  |  1e-06  |  yes  |  p045_p047  |  64  |  0  |  0.991  |  0.033  |  0.875  |  0.390  | 
ResNet50  |  0  |  Adam  |  0.0001  |  yes  |  p014_p039  |  16  |  0  |  0.903  |  0.312  |  0.912  |  0.260  | 


In [14]:
y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)

4983/4983 [==============================] - 2369s  


In [15]:
l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    line = y_pred[i]
    line = line.tolist()
    line.insert(0,name)
    l.append( line )

l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df = df.sort_values(by='img')
df.to_csv('2_ResNet50_400_300_p014_p039_l0.csv', index=None, float_format='%.3f')
print("submission saved")

submission saved


In [16]:
pre_submission = pd.read_csv('2_ResNet50_400_300_p014_p039_l0.csv')
clip_submission = pre_submission.drop('img',axis =1)
clip_submission = clip_submission.clip(0.001,1)
clip_submission.insert(0, 'img', pre_submission['img'])
clip_submission.to_csv('clipped_2_ResNet50_400_300_p014_p039_l0.csv', index=False)